In [1]:
import pandas as pd
import matplotlib.pylab as plt
import missingno as msno
from matplotlib import pyplot as plt
import statsmodels.api as sm

import sys
import re
import missingno as msno
from functools import reduce

import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode

from sklearn.svm import SVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier


from PIL import Image
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rc('font', family='AppleGothic')
mpl.rc('axes', unicode_minus=False)
plt.rcParams['axes.unicode_minus'] = False
from IPython.display import set_matplotlib_formats
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999

import datetime

In [2]:
data = pd.read_csv("total.csv")
data.head(2)

,Unnamed: 0,Organization Name,Organization Name URL,Industries,Headquarters Location,CB Rank (Company),Estimated Revenue Range,Operating Status,Founded Date,Founded Date Precision,Number of Articles,Industry Groups,Number of Founders,Number of Employees,Number of Funding Rounds,Funding Status,Last Funding Date,Last Funding Amount,Last Funding Amount Currency,Last Funding Amount Currency (in USD),Last Funding Type,Last Equity Funding Amount,Last Equity Funding Amount Currency,Last Equity Funding Amount Currency (in USD),Last Equity Funding Type,Total Equity Funding Amount,Total Equity Funding Amount Currency,Total Equity Funding Amount Currency (in USD),Total Funding Amount,Total Funding Amount Currency,Total Funding Amount Currency (in USD),Top 5 Investors,Number of Lead Investors,Number of Investors,Number of Events,G2 Stack - Total Products Active,IPqwery - Patents Granted,IPqwery - Trademarks Registered,Announced Date,Announced Date Precision,IPO Status,IPO Date,Acquisition Status,Closed Date,Closed Date Precision,Exit Date,Exit Date Precision,Website,Number of Exits,Headquarters Regions
0,0,BrikPay,https://www.crunchbase.com/organization/brikpay,"Artificial Intelligence, Construction, Software","Claymont, Delaware, United States",NaN,NaN,Active,2020-07-20,day,NaN,"Artificial Intelligence, Data and Analytics, R...",NaN,1-10,1,Seed,2020-07-20,1000000.0,USD,1000000.0,Seed,1000000.0,USD,1000000.0,Seed,1000000.0,USD,1000000.0,1000000.0,USD,1000000.0,Di Bartolomeo Holdings LLC,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Private,NaN,NaN,NaN,NaN,NaN,NaN,https://Brikpay.io,NaN,"Greater Philadelphia Area, East Coast, Souther..."
1,1,CAPITEVA,https://www.crunchbase.com/organization/capiteva,"Advertising, Consulting, Information Technolog...","Delhi, Delhi, India","45,612",NaN,Active,2020-07-06,day,NaN,"Advertising, Apps, Information Technology, Int...",1.0,11-50,1,Seed,2020-07-06,1200.0,USD,1200.0,Pre-Seed,1200.0,USD,1200.0,Pre-Seed,1200.0,USD,1200.0,1200.0,USD,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private,NaN,NaN,NaN,NaN,NaN,NaN,http://www.capiteva.com,NaN,Asia-Pacific (APAC)


In [3]:
import industry2

data =industry2.industry()

In [4]:
data["Acquisition Status"] = data["Acquisition Status"].fillna(0)
data["Acquisition Status"][data["Acquisition Status"] !=0] = 1
data["Funding Status"][data["IPO Status"]== "Public"] = "IPO"
data["Funding Status"][data["IPO Status"]== "Delisted"] = "IPO"
data["Funding Status"][data["Acquisition Status"]== 1] = "M&A"
data= data.drop(list(data["Founded Date"].sort_values(ascending = True).head(1965).index))
#data= data.drop(list(data["Founded Date"].sort_values(ascending = True).head(10565).index))

In [5]:
data["Financial_Services"][data["Financial_Services"] == True] =1
data["Financial_Services"][data["Financial_Services"] == False] =0
data["Financial_Services_Tech"][data["Financial_Services_Tech"] == True] =1
data["Financial_Services_Tech"][data["Financial_Services_Tech"] == False] =0
data["Information_Technology"][data["Information_Technology"] == True] =1
data["Information_Technology"][data["Information_Technology"] == False] =0
data["Software"][data["Software"] == True] =1
data["Software"][data["Software"] == False] =0
data["Hardware"][data["Hardware"] == True] =1
data["Hardware"][data["Hardware"] == False] =0
data["Commerce_and_Shopping"][data["Commerce_and_Shopping"] == True] =1
data["Commerce_and_Shopping"][data["Commerce_and_Shopping"] == False] =0
data["Media_Entertainment"][data["Media_Entertainment"] == True] =1
data["Media_Entertainment"][data["Media_Entertainment"] == False] =0
data["Health_care"][data["Health_care"] == True] =1
data["Health_care"][data["Health_care"] == False] =0
data["Internet_Services"][data["Internet_Services"] == True] =1
data["Internet_Services"][data["Internet_Services"] == False] =0
data["Science_Engineering"][data["Science_Engineering"] == True] =1
data["Science_Engineering"][data["Science_Engineering"] == False] =0

In [6]:
data["Last Funding Date"] = data["Last Funding Date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
data["Founded Date"] = data["Founded Date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
data["IPO Date"] = data["IPO Date"].fillna("2200-01-01")
data["IPO Date"] = data["IPO Date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
data["Announced Date"] = data["Announced Date"].fillna("2200-01-01")
data["Announced Date"] = data["Announced Date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
data["Funding_Duration"]  = data["Last Funding Date"] - data["Founded Date"] 
data["Funding_Duration"] = data["Funding_Duration"].apply(lambda x: (x / np.timedelta64(1, 'D')))

In [7]:
data["Operating Status"][data["Operating Status"] != "Active"] = 0
data["Operating Status"][data["Operating Status"] == "Active"]  = 1

data["Closed Date"] = data["Closed Date"].fillna("0000-00-00")
data["Life_Date"] = data["Closed Date"][data["Closed Date"] != "0000-00-00"]
data["Life_Date"] = data["Closed Date"][data["Operating Status"] == 0]
data["Life_Date"][data["Operating Status"]== 1] = "2020-08-01"

In [8]:
data["Life_Date"] = data["Life_Date"].fillna("0000-00-00")

k = list(data["Life_Date"][data["Life_Date"] == "0000-00-00"].index)
data= data.drop(k)

In [9]:
data["Life_Date"] = data["Life_Date"].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
data["Life_Date"] = data["Life_Date"] - data["Founded Date"]
data["Life_Date"] = data["Life_Date"].apply(lambda x: (x / np.timedelta64(1, 'D')))

In [10]:
df_angel=pd.read_csv("angel_1000 (1).csv")
df_vc = pd.read_csv("vc_1000.csv")
df_accelator = pd.read_csv("accelator_1000.csv")

df_accelator = df_accelator[["Organization/Person Name", "Number of Investments", 'Number of Exits']]
df_vc = df_vc[["Organization/Person Name", "Number of Investments", 'Number of Exits']]
df_angel = df_angel[["Organization/Person Name", "Number of Investments", 'Number of Exits']]

df_accelator["Organization/Person Name"] = df_accelator["Organization/Person Name"].apply(lambda x: x.replace(" ",""))
df_accelator["Number of Investments"] = df_accelator["Number of Investments"].apply(lambda x: int(str(x).replace(",","")))
df_accelator["Number of Exits"] = df_accelator["Number of Exits"].fillna(0)
df_accelator["Number of Exits"] = df_accelator["Number of Exits"].apply(lambda x: (str(x).replace(",","")))
df_accelator["Number of Exits"] = df_accelator["Number of Exits"].apply(lambda x: int(str(x).replace(".","")))

df_vc["Organization/Person Name"] = df_vc["Organization/Person Name"].apply(lambda x: x.replace(" ",""))
df_vc["Number of Investments"] = df_vc["Number of Investments"].apply(lambda x: int(str(x).replace(",","")))
df_vc["Number of Exits"] = df_vc["Number of Exits"].fillna(0)
df_vc["Number of Exits"] = df_vc["Number of Exits"].apply(lambda x: (str(x).replace(",","")))
df_vc["Number of Exits"] = df_vc["Number of Exits"].apply(lambda x: int(str(x).replace(".","")))

df_angel["Organization/Person Name"] = df_angel["Organization/Person Name"].apply(lambda x: x.replace(" ",""))
df_angel["Number of Investments"] = df_angel["Number of Investments"].apply(lambda x: int(str(x).replace(",","")))
df_angel["Number of Exits"] = df_angel["Number of Exits"].fillna(0)
df_angel["Number of Exits"] = df_angel["Number of Exits"].apply(lambda x: (str(x).replace(",","")))
df_angel["Number of Exits"] = df_angel["Number of Exits"].apply(lambda x: int(str(x).replace(".","")))

In [11]:
angel_list=str(list(df_angel["Organization/Person Name"]))
vc_list = str(list(df_vc["Organization/Person Name"]))
accelator_list = str(list(df_accelator["Organization/Person Name"]))

accelator_list =accelator_list.replace('\'', '')
accelator_list =accelator_list.replace('\n','')
accelator_list =accelator_list.replace(', ', '|' )
accelator_list =accelator_list.replace('[', '')
accelator_list =accelator_list.replace(']', '')
accelator_list =accelator_list.replace(' ', '')

vc_list =vc_list.replace('\'', '')
vc_list =vc_list.replace('\n','')
vc_list =vc_list.replace(', ', '|' )
vc_list =vc_list.replace('[', '')
vc_list =vc_list.replace(']', '')
vc_list =vc_list.replace(' ', '')

angel_list =angel_list.replace('\'', '')
angel_list =angel_list.replace('\n','')
angel_list =angel_list.replace(', ', '|' )
angel_list =angel_list.replace('[', '')
angel_list =angel_list.replace(']', '')
angel_list =angel_list.replace(' ', '')

data["Angel"] =data['Top 5 Investors'].str.contains(angel_list)
data["Angel"]=data["Angel"].fillna(0)

data["VC"] =data['Top 5 Investors'].str.contains(vc_list)
data["VC"]=data["VC"].fillna(0)

data["ACL"] =data['Top 5 Investors'].str.contains(accelator_list)
data["ACL"]=data["ACL"].fillna(0)

angel_list = set(angel_list.split("|"))
accelator_list = set(accelator_list.split("|"))
vc_list = set(vc_list.split("|"))

vc_invest =  dict(zip(df_vc["Organization/Person Name"], df_vc["Number of Investments"]))
angel_invest =  dict(zip(df_angel["Organization/Person Name"], df_angel["Number of Investments"]))
accelator_invest =  dict(zip(df_accelator["Organization/Person Name"], df_accelator["Number of Investments"]))
vc_exit=  dict(zip(df_vc["Organization/Person Name"], df_vc[ "Number of Exits"]))
angel_exit =  dict(zip(df_angel["Organization/Person Name"], df_angel["Number of Exits"]))
accelator_exit =  dict(zip(df_accelator["Organization/Person Name"], df_accelator["Number of Exits"]))

In [12]:
data['Top 5 Investors'] = data['Top 5 Investors'].apply(lambda x: str(x).replace(" ",""))
data['Top 5 Investors'][data['Top 5 Investors']== "nan"] = "asdf"

data['Angel'] = data['Top 5 Investors'].apply(lambda x: set(x.split(","))&angel_list)
data['ACL'] = data['Top 5 Investors'].apply(lambda x: set(x.split(","))&accelator_list)
data['VC'] = data['Top 5 Investors'].apply(lambda x: set(x.split(","))&vc_list)

data['Angel_Invest'] = data['Angel'].apply(lambda x: sum(list(map(angel_invest.get, list(x)))))
data['ACL_Invest'] = data['ACL'].apply(lambda x: sum(list(map(accelator_invest.get, list(x)))))
data['VC_Invest'] = data['VC'].apply(lambda x: sum(list(map(vc_invest.get, list(x)))))

data['Angel_Exit'] = data['Angel'].apply(lambda x: sum(list(map(angel_exit.get, list(x)))))
data['ACL_Exit'] = data['ACL'].apply(lambda x: sum(list(map(accelator_exit.get, list(x)))))
data['VC_Exit'] = data['VC'].apply(lambda x: sum(list(map(vc_exit.get, list(x)))))

In [13]:
# Fillna Mode 
print(len(data[data["Number of Founders"].isnull()==True]))

data["Number of Founders"] = data["Number of Founders"].fillna(1)

print(len(data[data['Number of Employees'].isnull()==True]))
data.loc[data['Number of Employees'] == "1-10",'Number of Employees']=1
data.loc[data['Number of Employees'] =="11-50",'Number of Employees']=2
data.loc[data['Number of Employees'] =="51-100",'Number of Employees']=3
data.loc[data['Number of Employees'] =="101-250",'Number of Employees']=4
data.loc[data['Number of Employees'] =="251-500",'Number of Employees']=5
data.loc[data['Number of Employees'] =="501-1000",'Number of Employees']=6
data.loc[data['Number of Employees'] =="1001-5000",'Number of Employees']=7
data.loc[data['Number of Employees'] =="5001-10000",'Number of Employees']=8
data.loc[data['Number of Employees'] =="10001+",'Number of Employees']=9

data['Number of Employees'] = data['Number of Employees'].fillna(1)
data['Last Funding Amount'] = data['Last Funding Amount'].fillna(1000)

print(len(data[data["Estimated Revenue Range"].isnull()==True]))
data.loc[data["Estimated Revenue Range"] == "Less than $1M" ,"Estimated Revenue Range"] = 1
data.loc[data["Estimated Revenue Range"] == "$1M to $10M" ,"Estimated Revenue Range"] = 2
data.loc[data["Estimated Revenue Range"] == "$10M to $50M" ,"Estimated Revenue Range"] = 3
data.loc[data["Estimated Revenue Range"] == "$50M to $100M" ,"Estimated Revenue Range"] = 4
data.loc[data["Estimated Revenue Range"] == "$100M to $500M" ,"Estimated Revenue Range"] = 5
data.loc[data["Estimated Revenue Range"] == "$500M to $1B" ,"Estimated Revenue Range"] = 6
data.loc[data["Estimated Revenue Range"] == "$1B to $10B" ,"Estimated Revenue Range"] = 7
data.loc[data["Estimated Revenue Range"] == "$10B+" ,"Estimated Revenue Range"] = 8

data["Estimated Revenue Range"]=data["Estimated Revenue Range"].fillna(1)

17728
2219
34263


In [14]:
data.columns = data.columns.str.replace(' ', '')
data.columns = data.columns.str.replace('-', '')
data.columns = data.columns.str.replace('_', '')

data["CBRank"]=data["CBRank(Company)"]
cb_index =list(data["CBRank"] [data["CBRank"].isnull()].index)
data = data.drop(cb_index)
data["CBRank"] = data["CBRank"].apply(lambda x: int(str(x).replace(",", "")))

In [15]:
#Fillna 1 & 0

print("NumberofArticles: {}".format(len(data[data['NumberofArticles'].isnull()==True])))
data["NumberofArticles"] = data["NumberofArticles"].fillna(0)
data["NumberofArticles"] = data["NumberofArticles"].apply(lambda x: str(x).translate(str.maketrans(',.', '  ')))
data["NumberofArticles"] = data["NumberofArticles"].apply(lambda x: int(str(x).replace(" ","")))
# data["NumberofArticles"][data["NumberofArticles"]>0] = 1

print("IPqweryPatentsGranted: {}".format(len(data[data['IPqweryPatentsGranted'].isnull()==True])))
data["IPqweryPatentsGranted"] = data["IPqweryPatentsGranted"].fillna(0)
data["IPqweryPatentsGranted"] = data["IPqweryPatentsGranted"].apply(lambda x: str(x).translate(str.maketrans(',.', '  ')))
data["IPqweryPatentsGranted"] = data["IPqweryPatentsGranted"].apply(lambda x: int(str(x).replace(" ","")))
#data["IPqweryPatentsGranted"][data["IPqweryPatentsGranted"]>0] = 1

print("G2StackTotalProductsActive: {}".format(len(data[data['G2StackTotalProductsActive'].isnull()==True])))
data["G2StackTotalProductsActive"] = data["G2StackTotalProductsActive"].fillna(0)
data["G2StackTotalProductsActive"] = data["G2StackTotalProductsActive"].apply(lambda x: str(x).translate(str.maketrans(',.', '  ')))
data["G2StackTotalProductsActive"] = data["G2StackTotalProductsActive"].apply(lambda x: int(str(x).replace(" ","")))
#data["G2StackTotalProductsActive"][data["G2StackTotalProductsActive"]>0] = 1

print("NumberofLeadInvestors: {}".format(len(data[data['NumberofLeadInvestors'].isnull()==True])))
data["NumberofLeadInvestors"] = data["NumberofLeadInvestors"].fillna(0)
# data["NumberofLeadInvestors"][data["NumberofLeadInvestors"]>0] = 1

print("NumberofEvents: {}".format(len(data[data['NumberofEvents'].isnull()==True])))
data['NumberofEvents'] = data['NumberofEvents'].fillna(0)
# data['NumberofEvents'][data['NumberofEvents']>0] = 1

print("IPqweryTrademarksRegistered: {}".format(len(data[data['IPqweryTrademarksRegistered'].isnull()==True])))
data["IPqweryTrademarksRegistered"] = data["IPqweryTrademarksRegistered"].fillna(0)
data["IPqweryTrademarksRegistered"] = data["IPqweryTrademarksRegistered"].apply(lambda x: str(x).translate(str.maketrans(',.', '  ')))
data["IPqweryTrademarksRegistered"] = data["IPqweryTrademarksRegistered"].apply(lambda x: int(str(x).replace(" ","")))
# data["IPqweryTrademarksRegistered"][data["IPqweryTrademarksRegistered"]>0]=1

print("NumberofInvestors: {}".format(len(data[data['NumberofInvestors'].isnull()==True])))
data['NumberofInvestors'] = data['NumberofInvestors'].fillna(1)

NumberofArticles: 35723
IPqweryPatentsGranted: 51095
G2StackTotalProductsActive: 35417
NumberofLeadInvestors: 39793
NumberofEvents: 61622
IPqweryTrademarksRegistered: 51095
NumberofInvestors: 17798


In [16]:
# data["HeadquartersLocation"] = data["HeadquartersLocation"].fillna(0)
# data["HeadquartersLocation"]= data["HeadquartersLocation"].str.contains('California').astype(int)

X = data[["TotalFundingAmount",'CBRank' ,"NumberofFundingRounds", "FundingDuration","NumberofFounders","NumberofInvestors","LastFundingAmount",
          'AngelInvest','ACLInvest','VCInvest','AngelExit','ACLExit','VCExit',"NumberofEvents","NumberofLeadInvestors","NumberofArticles","LifeDate",
          "IPqweryTrademarksRegistered","IPqweryPatentsGranted","G2StackTotalProductsActive",'FinancialServicesTech','FinancialServices','InformationTechnology',
          'Software', 'Hardware','CommerceandShopping', 'MediaEntertainment', 'Healthcare', 'InternetServices', 'ScienceEngineering',"NumberofEmployees",
          "EstimatedRevenueRange",'FundingStatus']]

In [17]:
X["FundingStatus"][X["FundingStatus"] == 'IPO'] = "Success"
X["FundingStatus"][X["FundingStatus"] == 'M&A'] = "Success"
# X["FundingStatus"][X["FundingStatus"] == "Early Stage Venture"] = "Venture"
# X["FundingStatus"][X["FundingStatus"] == "Late Stage Venture"] = "Venture"
X = X.drop(list(data[data["FundingStatus"]=="Private Equity"].index))
# X = X.drop(list(data[data["FundingStatus"]=="IPO"].index))

X = X.dropna()

X["FundingStatus"] [X["FundingStatus"] != "Success"] =0
X["FundingStatus"] [X["FundingStatus"] == "Success"] =1
X["FundingStatus"]=X["FundingStatus"].astype("int")

In [18]:
robust_scaler = RobustScaler()
rb = robust_scaler.fit(X.iloc[:,:20])
X.iloc[:,:20] = robust_scaler.transform(X.iloc[:,:20])

X.describe()

,TotalFundingAmount,CBRank,NumberofFundingRounds,FundingDuration,NumberofFounders,NumberofInvestors,LastFundingAmount,AngelInvest,ACLInvest,VCInvest,AngelExit,ACLExit,VCExit,NumberofEvents,NumberofLeadInvestors,NumberofArticles,LifeDate,IPqweryTrademarksRegistered,IPqweryPatentsGranted,G2StackTotalProductsActive,FinancialServicesTech,FinancialServices,InformationTechnology,Software,Hardware,CommerceandShopping,MediaEntertainment,Healthcare,InternetServices,ScienceEngineering,NumberofEmployees,EstimatedRevenueRange,FundingStatus
count,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000,74409.000000
mean,7.098804,0.196783,0.145083,0.328161,0.729979,0.432121,12.392526,2.226572,182.420379,165.868900,5.071295,158.348318,346.141327,0.652972,0.855837,7.424064,0.235803,12.721821,13.846631,0.340776,0.068634,0.096023,0.348076,0.454233,0.291054,0.145641,0.217326,0.174011,0.230805,0.159147,2.039982,1.470481,0.209934
std,395.531728,1.162149,0.913580,1.059717,0.996923,1.351404,751.992454,16.207832,687.615050,468.782915,41.222327,618.507345,1047.065032,4.001916,1.261036,392.740410,0.991090,108.220595,212.110311,0.812681,0.252833,0.294625,0.476364,0.497904,0.454251,0.352748,0.412429,0.379122,0.421351,0.365816,1.370606,0.867943,0.407264
min,-0.170940,-0.872823,-0.500000,-3.072000,-1.000000,-0.666667,-0.157233,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.166667,-1.418949,0.000000,0.000000,-0.235294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,-0.145299,-0.462239,-0.500000,-0.375333,0.000000,-0.333333,-0.135220,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.166667,-0.399781,0.000000,0.000000,-0.235294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000
75%,0.854701,0.537761,0.500000,0.624667,1.000000,0.666667,0.864780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.833333,0.600219,1.000000,0.000000,0.764706,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000
max,84957.094017,15.181079,16.500000,6.444667,17.000000,35.333333,156289.150943,429.000000,7098.000000,5237.000000,1300.000000,5620.000000,13110.000000,435.000000,23.000000,77353.500000,4.600767,8830.000000,25717.000000,61.947059,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000,8.000000,1.000000


In [19]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import IsolationForest

In [20]:
df_X = X.iloc[:,:-1]
df_y = X.iloc[:,[-1]]

k =list(df_y["FundingStatus"].apply(lambda x: int(x)))
neg, pos = np.bincount(k)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(total, pos, 100 * pos / total))

# le = LabelEncoder()
# le.fit(df_X['Industries'])
# df_X['Industries'] = le.transform(df_X['Industries'])

Examples:
    Total: 74409
    Positive: 15621 (20.99% of total)



In [21]:
from imblearn.combine import *
from imblearn.over_sampling import *

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

X_samp, y_samp = SMOTEENN(random_state=0).fit_sample(X_train, y_train)

In [42]:
# Random Forest

clf = RandomForestClassifier(criterion= "entropy", random_state=0).fit(X_train, y_train)

y_pred_t = clf.predict(X_test)

print(classification_report(np.array(y_test), y_pred_t))

              precision    recall  f1-score   support

           0       0.89      0.96      0.92     11731
           1       0.78      0.54      0.64      3151

    accuracy                           0.87     14882
   macro avg       0.84      0.75      0.78     14882
weighted avg       0.87      0.87      0.86     14882



In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
import xgboost as xgb



model2 = DecisionTreeClassifier(random_state=0)
model3 =GaussianNB()
model4 = KNeighborsClassifier(n_neighbors = 2)
ensemble = VotingClassifier(estimators=[('dtc', model2), ('qda',model3), ('knc', model4)], voting='soft')

ensemble.fit(X_train, y_train)
y_pred_t = ensemble.predict(X_test)
print(classification_report(np.array(y_test), y_pred_t))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90     11731
           1       0.66      0.47      0.55      3151

    accuracy                           0.84     14882
   macro avg       0.76      0.70      0.72     14882
weighted avg       0.82      0.84      0.82     14882



In [35]:
from sklearn.ensemble import BaggingClassifier

model2 = BaggingClassifier(DecisionTreeClassifier(),n_estimators=100, random_state=0).fit(X_train, y_train)
y_pred_t = model2.predict(X_test)
print(classification_report(np.array(y_test), y_pred_t))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     11731
           1       0.76      0.56      0.65      3151

    accuracy                           0.87     14882
   macro avg       0.83      0.76      0.78     14882
weighted avg       0.86      0.87      0.86     14882



In [29]:
model2 = xgb.XGBClassifier().fit(X_train, y_train)
y_pred_t = model2.predict(X_test)
print(classification_report(np.array(y_test), y_pred_t))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     11731
           1       0.76      0.57      0.65      3151

    accuracy                           0.87     14882
   macro avg       0.83      0.76      0.79     14882
weighted avg       0.86      0.87      0.86     14882



In [ ]:
importance = clf.feature_importances_
idx = X_train.columns
pd.DataFrame(importance, index =X_train.columns, columns=["Feature_importance"]).sort_values(["Feature_importance"], ascending = False)
plt.barh(idx,importance, height = 0.8,align='center',alpha=0.8)
plt.show()

In [ ]:
asd

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from tensorflow import keras

In [ ]:
X_train2 = np.array(X_train).astype("float")
y_train2 = to_categorical(y_train, num_classes=2)
X_test2 = np.array(X_test).astype("float")
y_test2 = to_categorical(y_test, num_classes=2)

model = keras.models.Sequential([
    keras.layers.Dense(256, activation='relu', input_dim=33),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2, activation='sigmoid')
])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

hist = model.fit(X_train2, y_train2, epochs=100, batch_size=1000)

In [ ]:
loss_and_metrics = model.evaluate(X_test2,y_test2)
predictions = model.predict(X_test2)
print('loss_and_metrics : ' + str(loss_and_metrics))

In [ ]:
len(y_test2)

In [ ]:
for i in range(14882):
    if y_test2[i][0]==1:
        y_test2[i]=0
    elif y_test2[i][1]==1:
        y_test2[i]=1
#     elif y_test[i][2]==1:
#         y_test[i]=2
#     elif y_test[i][3]==1:
#         y_test[i]=3
#     elif y_test[i][4]==1:
#         y_test[i]=4  

In [ ]:
a = []
b = []
for i in range(14882):
    a.append(np.argmax(predictions[i]))
    b.append((y_test2[i][0]))
c = np.array(a)[:, np.newaxis]
d = np.array(b)[:, np.newaxis]
print(classification_report(c,d))